In [13]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5f7c48bd1fb96588cd724b3f659228cef1a3bfe692ca019306d9a969515c558731ab8f7318033b5e2e9ad52537fccb9288a0bfa6c490cb1298u'

import parse

In [14]:
import re

In [15]:
from aocd.models import Puzzle


def parses(input):
    costs = []
    # ore=0, clay=1, obsidian=2, geode=3 (robot, resource)
    idxs = np.array([(0,0),(1,0),(2,0),(2,1),(3,0),(3,2)])
    for line in input.strip().split('\n'):
        nums = re.findall(r'\d+',line)[1:]
        cost = np.zeros((4,4), dtype=np.int32)
        cost[tuple(idxs.T)] = np.array(nums)
        costs.append(cost)
    return costs
    

puzzle = Puzzle(year=2022, day=19)
data = parses(puzzle.input_data)

In [16]:
sample = parses("""Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.""")

In [17]:
def max_geodes(blueprint, mins):
    def H(mins, resources, robots):
        return tuple([mins]+resources.tolist()+robots.tolist())
    GEODE = 3
    max_cost = blueprint.max(axis=0) 
    max_cost[GEODE] = mins 
    
    stack = [(mins, np.array([0,0,0,0], dtype=np.int32), np.array([1,0,0,0], dtype=np.int32))]
    
    best_geodes = 0
    visited = set([H(*stack[0])])
    
    while stack:
        remaining, resources, robots = stack.pop()
        best_geodes = max(best_geodes, resources[GEODE])
        if remaining == 0:
            continue
        
        # We should drop a branch that cannot generate more geodes that the best case so far
        upper_bound = resources[GEODE] + robots[GEODE] * remaining + remaining*(remaining-1)/2
        if upper_bound <= best_geodes:
            continue
        
        for robot, cost in enumerate(blueprint):
            new_robots = robots.copy()
            new_robots[robot] += 1
            # 1: We should not have more resource supplying robots than what we can spend in a turn
            # 2: We can't wait to make something we are not collecting resources for
            # 3: We should not build a robot we could have build the previous iteration
            if (new_robots > max_cost).any() or not (robots >= (cost > 0)).all() or (resources-cost >= cost).all():
                continue
    
            missing = cost - resources
            # We have N turns to collect missing resources and 1 turn to make the robot
            wait = 1 + int(max(np.ceil(missing[missing > 0] / robots[missing>0]), default=0))
            
            new_resources = resources + robots * wait - cost
            
            new = (remaining-wait, new_resources, new_robots)
            if wait <= remaining and (h := H(*new)) not in visited:
                visited.add(h)
                stack.append(new)
        
        # no-op till the end
        best_geodes = max(best_geodes, (resources+robots*remaining)[GEODE])

    return best_geodes



In [103]:
sample[0]

array([[ 4,  0,  0,  0],
       [ 2,  0,  0,  0],
       [ 3, 14,  0,  0],
       [ 2,  0,  7,  0]], dtype=int32)

In [161]:
from heapq import heappush, heappop

def max_geodes(blueprint, mins):
    blueprint = blueprint.astype(int).tolist()
    max_cost = tuple(max(blueprint[j][i] for j in range(4)) for i in range(3)) # no geode cost

    blueprint = [tuple(row[:-1]) for row in blueprint] # no geode cost
    
    upper_geodes = lambda x: (x+1)*x/2
    
    heap = [(-upper_geodes(mins-1), 0, mins, (1,0,0), (0,0,0))]
    visited = set(heap)
    
    best_geodes = 0
    
    while heap:
        _, geodes, remaining, robots, resources = heappop(heap)
        if remaining == 0:
            return geodes
        upper = geodes + upper_geodes(remaining)
        if upper <= best_geodes:
            continue
        best_geodes = max(best_geodes, geodes)
            
        for robot, cost in enumerate(blueprint):
            # if we have enough robots skip
            if robot < 3 and robots[robot] >= max_cost[robot]:
                continue
            # if we can't make it
            if any( req > 0 and rob == 0 for req, rob in zip(cost, robots)):
                continue
            missing = tuple( c-r for c, r in zip(cost, resources))
            wait = 1 + int( max( (math.ceil(req/rob)  for req, rob in zip(missing, robots) if req > 0), default=0))
            # not enough time
            if wait >= remaining:
                continue
            new_resources = tuple([-now+wait*prod for now, prod in zip(missing, robots)])
            if robot < 3:
                new_robots = tuple([r+(i==robot) for i, r in enumerate(robots)])
                new_geodes = geodes
            else:
                new_robots = robots
                new_geodes = geodes + remaining-wait
            
            estimate = new_geodes + upper_geodes(remaining-wait) 
            state = (-estimate, new_geodes, remaining-wait, new_robots, new_resources)
            if state not in visited:
                visited.add(state)
                heappush(heap, state)
        
        # no-op
        state = (-geodes, geodes, 0, robots, resources)
        if state not in visited:
            heappush(heap, state)

    return best_geodes

In [160]:
max_geodes_astar(sample[0], 24)

9

In [91]:
sample

[array([[ 4,  0,  0,  0],
        [ 2,  0,  0,  0],
        [ 3, 14,  0,  0],
        [ 2,  0,  7,  0]], dtype=int32),
 array([[ 2,  0,  0,  0],
        [ 3,  0,  0,  0],
        [ 3,  8,  0,  0],
        [ 3,  0, 12,  0]], dtype=int32)]

ValueError: max() arg is an empty sequence

In [73]:
# def max_geodes(blueprint, mins): 
#     states = [(np.array([0,0,0,0], dtype=np.int32), np.array([1,0,0,0], dtype=np.int32))]
#     keep = 10_000
    
#     for t in range(mins):
#         new_states = []
#         for resources, robots in states:
#             new_states.append((resources+robots, robots)) # no-op
#             for robot, cost in enumerate(blueprint):
#                 if (resources >= cost).all():
#                     new_robots = robots + np.array([i==robot for i in range(4)])
#                     new_states.append((resources-cost+robots, new_robots))
#         states = sorted(new_states, key=lambda x: tuple((x[0]+x[1])[::-1]))[-keep:]

#     return max(robots[3] for resources, robots in states)

In [162]:
def solve_a(data):
    qs = [max_geodes(x, 24) for x in data]
    return sum((i * q for i, q in enumerate(qs, start=1)))

In [163]:
import math
def solve_b(data):
    return math.prod([max_geodes(x, 32) for x in data[:3]])

In [164]:
# GT = [5, 0, 2, 10, 3, 6, 0, 11, 3, 8, 1, 0, 1, 0, 15, 0, 0, 12, 0, 2, 1, 0, 3, 14, 1, 1, 2, 0, 9, 1]
# np.where(np.array(ps) != np.array(GT))

In [165]:
solve_a(sample)

33

In [166]:
solve_b(sample)

3472

In [167]:
%%time
solve_a(data)

CPU times: user 5.98 s, sys: 74.6 ms, total: 6.05 s
Wall time: 6.1 s


1624

In [168]:
%%time
solve_b(data)

CPU times: user 8.34 s, sys: 145 ms, total: 8.49 s
Wall time: 8.56 s


12628

In [ ]:
solve_b(sample)

In [175]:
max_geodes(data[9], 24)

8

In [176]:
GT[9]

8

In [177]:
max_geodes(data[23], 24)

14

In [169]:
GT[23]

14

In [ ]:
1624

In [126]:
def solve_b(data):
    return math.prod(parallel_max_geodes(data[:3], 32))

In [127]:
solve_b(sample)

3410

In [128]:
solve_b(data)

11070

In [ ]:
12628

In [134]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2022, day=19)

def parses(data, as_tuples=True):
    blueprints = []
    for line in data.split('\n'):
        _, *recipes = line.split('Each')
        blueprints.append({})
        for recipe in recipes:
            recipe = recipe.strip(' .')
            output, inputs = parse.parse('{} robot costs {}', recipe).fixed
            inputs = [parse.parse('{:d} {}', input).fixed for input in inputs.split(' and ')]
            inputs = {a: n for n, a in inputs}
            blueprints[-1][output] = inputs
    if not as_tuples:
        return blueprints
    materials = ('ore', 'clay', 'obsidian', 'geode')
    data = []
    for blueprint in blueprints:
        data.append([])
        for robot in sorted(blueprint, key=materials.index):
            cost = [0 for _ in range(4)]
            for resource, count in  blueprint[robot].items():
                cost[materials.index(resource)] = count
            data[-1].append(tuple(cost))
    return data

        
data = parses(puzzle.input_data)

In [135]:
sample = parses("""Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.""")

In [136]:
import operator

In [137]:
# def max_geodes(blueprint, n=24):
#     add = lambda x, y: tuple(map(operator.add, x, y))
#     sub = lambda x, y: tuple(map(operator.sub, x, y))
#     gea = lambda x, y: all(map(operator.ge, x, y))
#     cache = {}
    
#     def dp(remaining, robots, resources):
#         if (remaining, robots, resources) not in cache:
#             if remaining == 0:
#                 geodes = resources[3]
#             else:
#                 geodes = 0
#                 for robot, recipe in enumerate(blueprint):
#                     if gea(resources, recipe):
#                         new_resources = sub(resources, recipe)
#                         new_resources = add(new_resources, robots)
#                         new_robots = add(robots, [i==robot for i in range(4)])
#                         geodes = max(geodes, dp(remaining-1, new_robots, new_resources))
#                 # no-op
#                 new_resources = add(resources, robots)
#                 geodes = max(geodes, dp(remaining-1, robots, new_resources))
#             cache[remaining, robots, resources] = geodes
#         return cache[remaining, robots, resources]
    
#     dp(n, (1,0,0,0), (0,0,0,0))
# #     return cache
#     return dp(n, (1,0,0,0), (0,0,0,0))


In [138]:
import math
def max_geodes(blueprint, n=24):
    add = lambda x, y: tuple(map(operator.add, x, y))
    sub = lambda x, y: tuple(map(operator.sub, x, y))
#     gea = lambda x, y: all(map(operator.ge, x, y))
    cache = {}       
    
    def dp(remaining, robots, resources):
        if (remaining, robots, resources) not in cache:
            if remaining == 0:
                geodes = resources[3]
            else:
                geodes = 0
                for robot, recipe in enumerate(blueprint):
                    feasible = all( prod >= (req > 0) for req, prod in zip(recipe, robots) )
                    
                    if feasible:
                        required = sub(recipe, resources)
                        waiting = 0
                        for req, prod in zip( sub(recipe, resources), robots):
                            if req > 0:
                                waiting = max(waiting, math.ceil(req/prod))
                        
                        waiting += 1
                        
                        if waiting <= remaining:
                            new_resources = add(resources, tuple((i*waiting for i in robots)))
                            new_resources = sub(new_resources, recipe)
                            new_robots = add(robots, [i==robot for i in range(4)])
                            geodes = max(geodes, dp(remaining-waiting, new_robots, new_resources))
                # no-op
                geodes = max(geodes, remaining*robots[-1])
            cache[remaining, robots, resources] = geodes
        return cache[remaining, robots, resources]
    
    return dp(n, (1,0,0,0), (0,0,0,0))


In [16]:
from concurrent.futures import ProcessPoolExecutor

In [139]:
def solve_a(data):
    with ProcessPoolExecutor(max_workers=len(data)) as executor:
        qualities = list(executor.map(max_geodes, data))
    return qualities
#     return sum((i * q for i, q in enumerate(qualities, start=1)))

In [140]:
solve_a(sample)

Process ForkProcess-81:
Traceback (most recent call last):
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/home/jjgo/.conda/envs/helix/lib/python3.9/multiprocessing/connection.py", line 384, in _recv
    chunk = read(han

KeyboardInterrupt: 

In [141]:
solve_a(data)

[5,
 0,
 2,
 10,
 3,
 6,
 0,
 11,
 3,
 8,
 1,
 0,
 1,
 0,
 15,
 0,
 0,
 12,
 0,
 2,
 1,
 0,
 3,
 14,
 1,
 1,
 2,
 0,
 9,
 1]

In [22]:
import time

In [25]:
X = sample[1]

for i in range(1,25):
    start = time.time()
    cache, res = max_geodes(X, i)
    taken = time.time() - start
    print(f'{i} - {taken:.2f}s - {res} - {len(cache)}')
    

1 - 0.00s - 0 - 1
2 - 0.00s - 0 - 1
3 - 0.00s - 0 - 2
4 - 0.00s - 0 - 3
5 - 0.00s - 0 - 5
6 - 0.00s - 0 - 9
7 - 0.00s - 0 - 16
8 - 0.00s - 0 - 30
9 - 0.00s - 0 - 55
10 - 0.00s - 0 - 99
11 - 0.00s - 0 - 207
12 - 0.01s - 0 - 463
13 - 0.01s - 0 - 1080
14 - 0.03s - 0 - 2612
15 - 0.06s - 0 - 6330
16 - 0.14s - 0 - 15035
17 - 0.33s - 0 - 34693
18 - 0.80s - 0 - 77733
19 - 1.77s - 1 - 179191
20 - 4.65s - 2 - 439361
21 - 11.07s - 4 - 1121276
22 - 29.92s - 6 - 2920815
23 - 80.75s - 9 - 7624972
24 - 209.96s - 12 - 19574502


In [107]:
import math
def max_geodes2(blueprint, thres, n=32):
    add = lambda x, y: tuple(map(operator.add, x, y))
    sub = lambda x, y: tuple(map(operator.sub, x, y))
#     gea = lambda x, y: all(map(operator.ge, x, y))
    cache = {}       
    
    def dp(remaining, robots, resources):
        if (remaining, robots, resources) not in cache:
            if remaining == 0:
                geodes = resources[3]
            else:
                geodes = 0
                for robot, recipe in enumerate(blueprint):
                    if n-remaining >= thres and robot in (0,1):
                        continue
                    feasible = all( prod >= (req > 0) for req, prod in zip(recipe, robots) )
                    
                    if feasible:
                        required = sub(recipe, resources)
                        waiting = 0
                        for req, prod in zip( sub(recipe, resources), robots):
                            if req > 0:
                                waiting = max(waiting, math.ceil(req/prod))
                        
                        waiting += 1
                        
                        if waiting <= remaining:
                            new_resources = add(resources, tuple((i*waiting for i in robots)))
                            new_resources = sub(new_resources, recipe)
                            new_robots = add(robots, [i==robot for i in range(4)])
                            geodes = max(geodes, dp(remaining-waiting, new_robots, new_resources))
                # no-op
                geodes = max(geodes, remaining*robots[-1])
            cache[remaining, robots, resources] = geodes
        return cache[remaining, robots, resources]
    
    return dp(n, (1,0,0,0), (0,0,0,0))


In [105]:
# max_geodes2(sample[0], 32)
# max_geodes2(sample[1], 32)

In [112]:
for thres in range(15,25):
    print(thres, max_geodes2(data[2], thres))

15 26
16 28
17 28
18 28
19 28
20 28
21 28


KeyboardInterrupt: 

In [113]:
41 * 11 * 28

12628

In [106]:
a = 
b = max_geodes2(data[1], 32)
c = max_geodes2(data[2], 32)

a * b * c

10332

In [30]:
7098
7280
8528
9184


()

In [68]:
import math
def max_geodes_trace(blueprint, n=24, opt=3):
    add = lambda x, y: tuple(map(operator.add, x, y))
    sub = lambda x, y: tuple(map(operator.sub, x, y))
    cache = {}       
    
    def dp(remaining, robots, resources):
        if (remaining, robots, resources) not in cache:
            if remaining == 0:
                geodes = (resources[opt], tuple())
            else:
                geodes = (0, tuple())
                for robot, recipe in enumerate(blueprint):
                    feasible = all( prod >= (req > 0) for req, prod in zip(recipe, robots) )
                    
                    if feasible:
                        required = sub(recipe, resources)
                        waiting = 0
                        for req, prod in zip( sub(recipe, resources), robots):
                            if req > 0:
                                waiting = max(waiting, math.ceil(req/prod))
                        
                        waiting += 1
                        
                    
                        if waiting <= remaining:
                            # buy robot
                            new_resources = add(resources, tuple((i*waiting for i in robots)))
                            new_resources = sub(new_resources, recipe)
                            new_robots = add(robots, [i==robot for i in range(4)])
                            x, trace = dp(remaining-waiting, new_robots, new_resources)
                            trace += ((robot, remaining-waiting+1, new_resources),)
                            geodes = max(geodes, (x, trace))
                # no-op
                geodes = max(geodes, (remaining*robots[opt], tuple()))
            cache[remaining, robots, resources] = geodes
        return cache[remaining, robots, resources]
    
    return dp(n, (1,0,0,0), (0,0,0,0))


In [70]:
T = 24
q, robots = max_geodes_trace(sample[0], T, 3)
print(q)
materials = ('ore', 'clay', 'obsidian', 'geode')
for what, when, avail in sorted(robots, key=lambda x: -x[1]):
    print(f"T={T-when} - {materials[what]} - {avail}")

9
T=2 - clay - (1, 0, 0, 0)
T=4 - clay - (1, 2, 0, 0)
T=6 - clay - (1, 6, 0, 0)
T=10 - obsidian - (2, 4, 0, 0)
T=11 - clay - (1, 7, 1, 0)
T=14 - obsidian - (1, 5, 4, 0)
T=17 - geode - (2, 17, 3, 0)
T=20 - geode - (3, 29, 2, 3)
T=21 - obsidian - (1, 19, 4, 5)
T=23 - geode - (1, 27, 3, 9)


In [71]:
T = 24
q, robots = max_geodes_trace(data[0], T, 3)
print(q)
materials = ('ore', 'clay', 'obsidian', 'geode')
for what, when, avail in sorted(robots, key=lambda x: -x[1]):
    print(f"T={T-when} - {materials[what]} - {avail}")

5
T=2 - ore - (1, 0, 0, 0)
T=4 - ore - (3, 0, 0, 0)
T=5 - ore - (4, 0, 0, 0)
T=6 - clay - (4, 0, 0, 0)
T=7 - clay - (4, 1, 0, 0)
T=8 - clay - (4, 3, 0, 0)
T=9 - clay - (4, 6, 0, 0)
T=10 - clay - (4, 10, 0, 0)
T=11 - clay - (4, 15, 0, 0)
T=12 - clay - (4, 21, 0, 0)
T=13 - obsidian - (6, 12, 0, 0)
T=14 - ore - (8, 19, 1, 0)
T=15 - obsidian - (11, 10, 2, 0)
T=16 - ore - (14, 17, 4, 0)
T=17 - obsidian - (18, 8, 6, 0)
T=18 - ore - (22, 15, 9, 0)
T=19 - geode - (27, 22, 3, 0)
T=20 - ore - (32, 29, 6, 1)
T=22 - geode - (46, 43, 3, 3)
T=23 - obsidian - (52, 34, 6, 5)


0
T=4 - clay - (1, 0, 0, 0)
T=8 - clay - (1, 4, 0, 0)
T=12 - clay - (1, 12, 0, 0)


In [76]:
T = 26
q, robots = max_geodes_trace(data[1], T, 3)
print(q)
materials = ('ore', 'clay', 'obsidian', 'geode')
for what, when, avail in sorted(robots, key=lambda x: -x[1]):
    print(f"T={T-when} - {materials[what]} - {avail}")

1
T=4 - ore - (1, 0, 0, 0)
T=7 - ore - (3, 0, 0, 0)
T=9 - clay - (5, 0, 0, 0)
T=10 - ore - (4, 1, 0, 0)
T=11 - clay - (4, 2, 0, 0)
T=12 - clay - (4, 4, 0, 0)
T=13 - clay - (4, 7, 0, 0)
T=14 - clay - (4, 11, 0, 0)
T=15 - clay - (4, 16, 0, 0)
T=16 - obsidian - (4, 8, 0, 0)
T=17 - clay - (4, 14, 1, 0)
T=18 - obsidian - (4, 7, 2, 0)
T=19 - ore - (4, 14, 4, 0)
T=20 - obsidian - (5, 7, 6, 0)
T=22 - obsidian - (11, 7, 12, 0)
T=24 - geode - (19, 21, 4, 0)
T=25 - obsidian - (20, 14, 8, 1)


In [74]:
data[1]

[(4, 0, 0, 0), (4, 0, 0, 0), (4, 14, 0, 0), (2, 0, 16, 0)]

In [75]:
T = 24
q, robots = max_geodes_trace(data[2], T, 3)
print(q)
materials = ('ore', 'clay', 'obsidian', 'geode')
for what, when, avail in sorted(robots, key=lambda x: -x[1]):
    print(f"T={T-when} - {materials[what]} - {avail}")

2
T=4 - ore - (1, 0, 0, 0)
T=7 - clay - (3, 0, 0, 0)
T=9 - clay - (3, 2, 0, 0)
T=11 - clay - (3, 6, 0, 0)
T=13 - clay - (3, 12, 0, 0)
T=15 - obsidian - (3, 8, 0, 0)
T=17 - obsidian - (3, 4, 2, 0)
T=19 - clay - (3, 12, 6, 0)
T=21 - geode - (4, 22, 2, 0)


In [ ]:
T = 24
q, robots = max_geodes_trace(data[2], T, 3)
print(q)
materials = ('ore', 'clay', 'obsidian', 'geode')
for what, when, avail in sorted(robots, key=lambda x: -x[1]):
    print(f"T={T-when} - {materials[what]} - {avail}")

In [64]:
data

[[(2, 0, 0, 0), (4, 0, 0, 0), (2, 16, 0, 0), (2, 0, 9, 0)],
 [(4, 0, 0, 0), (4, 0, 0, 0), (4, 14, 0, 0), (2, 0, 16, 0)],
 [(4, 0, 0, 0), (4, 0, 0, 0), (4, 12, 0, 0), (3, 0, 8, 0)],
 [(2, 0, 0, 0), (3, 0, 0, 0), (2, 14, 0, 0), (3, 0, 8, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (2, 15, 0, 0), (3, 0, 7, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (4, 8, 0, 0), (2, 0, 10, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (3, 16, 0, 0), (3, 0, 14, 0)],
 [(4, 0, 0, 0), (3, 0, 0, 0), (2, 7, 0, 0), (3, 0, 8, 0)],
 [(2, 0, 0, 0), (4, 0, 0, 0), (4, 17, 0, 0), (3, 0, 11, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (3, 6, 0, 0), (4, 0, 11, 0)],
 [(4, 0, 0, 0), (4, 0, 0, 0), (2, 9, 0, 0), (3, 0, 15, 0)],
 [(4, 0, 0, 0), (4, 0, 0, 0), (4, 14, 0, 0), (3, 0, 16, 0)],
 [(4, 0, 0, 0), (4, 0, 0, 0), (4, 8, 0, 0), (4, 0, 14, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (2, 19, 0, 0), (2, 0, 12, 0)],
 [(2, 0, 0, 0), (3, 0, 0, 0), (3, 9, 0, 0), (3, 0, 9, 0)],
 [(4, 0, 0, 0), (3, 0, 0, 0), (4, 19, 0, 0), (4, 0, 12, 0)],
 [(3, 0, 0, 0), (4, 0, 0, 0), (4, 17

In [27]:
# from pyrsistent import pmap
# from immutables import Map as pmap

# def max_geodes(blueprint, n=24):
#     cache = {}
    
#     def dp(remaining, robots, resources):
#         if (remaining, robots, resources) not in cache:
#             if remaining == 0:
#                 geodes = resources.get('geode',0)
#             else:
#                 geodes = 0
#                 for robot, recipe in blueprint.items():
#                     new_resources = pmap({material: resources.get(material,0)-cost for material, cost in recipe.items()})
#                     if any((r < 0 for r in new_resources.values())):
#                         continue
#                     new_resources = pmap({robot: count+new_resources.get(robot, 0) for robot, count in robots.items()})
#                     new_robots = robots.set(robot, 1+robots.get(robot,0))
#                     geodes = max(geodes, dp(remaining-1, new_robots, new_resources))
                    
#                 # no-op
#                 new_resources = pmap({robot: count+resources.get(robot, 0) for robot, count in robots.items()})
#                 geodes = max(geodes, dp(remaining-1, robots, new_resources))
        
#             cache[remaining, robots, resources] = geodes
#         return cache[remaining, robots, resources]
#     dp(n, pmap({'ore': 1}), pmap())
#     return dp(n, pmap({'ore': 1}), pmap())
# #     return dp(n, pmap(), pmap({'ore': 1}))